In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [5]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['CO[g/s]']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [8]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [9]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [10]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:.2f}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.02
MAPE: 23500927611.50%


GRID

In [11]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [12]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=2,  
    verbose=3,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 1012500 candidates, totalling 2025000 fits
Best: 0.039222 using {'learning_rate': 0.9, 'max_depth': 17, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 41, 'subsample': 0.5}
Best RMSE: 0.03922246887364486
Best MAPE: 28243907408.94867
Execution time: 58437.76297521591 s


In [29]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.03414291724288041
MAPE: 39526819740.62%


In [14]:
# export to excel
pd.DataFrame(grid.cv_results_).to_excel('GBR_GS_unscal_CO.xlsx', index=False)

random search

In [15]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=5,  
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.024542 using {'subsample': 0.6, 'n_estimators': 16, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 13, 'learning_rate': 0.9}
Best RMSE: 0.02454224949795181
Best MAPE: 24597187578.555626
Execution time: 16.614335775375366 s


In [28]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.025074231156944193
MAPE: 33891484332.92%


In [17]:
rs.cv_results_

{'mean_fit_time': array([0.27567363, 1.41032286, 1.09178982, 0.26426058, 0.52499962,
        0.03750067, 0.60312524, 1.21965809, 0.42217922, 0.28705697,
        2.51761031, 0.45330491, 0.31881166, 1.21989131, 1.32952456,
        0.21888614, 0.00312457, 0.13749905, 0.17529497, 1.24448233,
        0.33154154, 0.63164616, 0.1062501 , 0.39726911, 0.93797722,
        0.20937519, 0.02812414, 0.21875024, 0.43789582, 0.52852187,
        0.27852101, 1.14141054, 1.36016326, 0.09687552, 0.30351787,
        0.55976667, 0.07187595, 1.95472889, 0.03125005, 0.65039706,
        0.45014768, 0.36948714, 0.13164334, 0.50378413, 0.10646052,
        0.80351782, 0.78476677, 1.37625618, 0.23750038, 0.65039706,
        0.28164701, 0.14083886, 0.16562572, 1.66313977, 0.62890258,
        0.05937581, 0.80117798, 0.9882113 , 0.15312538, 0.37852411,
        0.87192698, 0.21250162, 0.14062538, 0.74130726, 0.38122211,
        0.98003244, 0.37301259, 0.67695084, 0.81531639, 0.11875067,
        0.25041809, 0.02922864,

In [18]:
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_learning_rate,...,std_test_RMSE,rank_test_RMSE,split0_test_MAPE,split1_test_MAPE,split2_test_MAPE,split3_test_MAPE,split4_test_MAPE,mean_test_MAPE,std_test_MAPE,rank_test_MAPE
0,0.275674,0.007938,0.006251,0.007655,0.7,41,6,5,2,0.9,...,0.002149,62,1.507643e+10,2.166136e+10,2.054068e+10,3.502022e+10,1.630390e+10,2.172052e+10,7.095930e+09,35
1,1.410323,0.041690,0.021302,0.007003,0.9,26,5,2,22,0.6,...,0.002116,20,4.394060e+09,1.847209e+10,1.587075e+10,2.543578e+10,2.014181e+10,1.686290e+10,6.975548e+09,88
2,1.091790,0.052633,0.012500,0.006250,0.5,46,2,5,14,0.9,...,0.001257,3,1.404947e+10,1.866046e+10,5.149916e+10,2.736264e+10,1.777059e+10,2.586846e+10,1.353798e+10,17
3,0.264261,0.022964,0.000000,0.000000,0.9,6,8,6,20,0.5,...,0.001875,64,9.820479e+09,2.103009e+10,1.480507e+10,2.773704e+10,1.357154e+10,1.739284e+10,6.306688e+09,75
4,0.525000,0.036444,0.009376,0.007655,0.6,16,6,5,23,0.2,...,0.002132,83,8.037332e+09,2.192793e+10,2.034932e+10,2.814912e+10,1.535559e+10,1.876386e+10,6.742148e+09,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.021475,0.023953,0.010539,0.007178,0.9,31,3,5,9,0.4,...,0.002062,61,8.298910e+09,1.848218e+10,1.836216e+10,2.506947e+10,1.479179e+10,1.700090e+10,5.472730e+09,85
96,0.619287,0.024680,0.018069,0.002638,0.5,21,8,9,17,0.1,...,0.001867,95,1.247212e+10,2.430231e+10,2.271146e+10,3.288260e+10,1.972792e+10,2.241928e+10,6.622714e+09,32
97,0.036196,0.007489,0.000000,0.000000,0.6,1,2,9,25,0.7,...,0.002231,58,1.505537e+10,3.585074e+10,4.366722e+10,4.927144e+10,2.837906e+10,3.444476e+10,1.199669e+10,12
98,0.691025,0.031239,0.008323,0.007060,0.9,16,9,8,13,0.4,...,0.001954,73,9.659299e+09,1.906933e+10,1.420130e+10,2.577722e+10,1.491036e+10,1.672350e+10,5.422630e+09,91


In [19]:
# export to excel
pd.DataFrame(rs.cv_results_).to_excel('GBR_RS_unscal_CO.xlsx', index=False)

BAYESSIAN

In [20]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample= (subsample), 
        min_samples_split=int(min_samples_split), 
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [21]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate': (0.01, 1.0),
        'n_estimators': (1, 51),
        'subsample': (0.5, 1.0),
        'min_samples_split': (2, 11),
        'min_samples_leaf': (1, 11),
        'max_depth': (1, 26),
    }
)

In [22]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -0.01897  | 0.5355    | 22.91     | 2.568     | 4.738     | 27.95     | 0.5259    |
| 2         | -0.01892  | 0.9397    | 21.57     | 4.524     | 9.03      | 35.58     | 0.9588    |
| 3         | -0.0184   | 0.6452    | 13.95     | 2.687     | 3.765     | 41.32     | 0.7727    |
| 4         | -0.01708  | 0.4309    | 25.37     | 3.083     | 5.71      | 13.86     | 0.6464    |
| 5         | -0.01651  | 0.6337    | 7.167     | 10.96     | 5.99      | 28.22     | 0.5776    |
| 6         | -0.01513  | 0.09894   | 11.27     | 4.21      | 3.976     | 21.4      | 0.686     |
| 7         | -0.01617  | 0.2937    | 22.91     | 6.679     | 8.977     | 31.87     | 0.6412    |
| 8         | -0.01887  | 0.0224    | 24.28     | 6.215     | 5.991     | 16.07     | 0.8412    |
| 9         | -0.019

In [23]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [24]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.7032957363528153, 'max_depth': 12.1808852204168, 'min_samples_leaf': 4.027384367661961, 'min_samples_split': 8.546300204803815, 'n_estimators': 32.751883059926286, 'subsample': 0.5176940128404173}


In [25]:
best_bs = GradientBoostingRegressor(
    learning_rate=(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=(best_params['subsample']), 
    min_samples_split=int(best_params['min_samples_split']), 
    min_samples_leaf=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.7032957363528153, max_depth=12,
                          min_samples_leaf=32, min_samples_split=8,
                          n_estimators=32, subsample=0.5176940128404173)

In [27]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.016311796038417415
MAPE: 23732661298.70%
